In [1]:
import csv
import numpy as np
import os
import pandas as pd

In [7]:
EDA_list = os.listdir('./KEMDy20/EDA_DATA/')
IBI_list = os.listdir('./KEMDy20/IBI_DATA/')
TEMP_list = os.listdir('./KEMDy20/TEMP_DATA/')

EDA_list = sorted(EDA_list)
IBI_list = sorted(IBI_list)
TEMP_list = sorted(TEMP_list)

if '.DS_Store' in EDA_list:
    EDA_list.remove('.DS_Store')

if '.DS_Store' in IBI_list:
    IBI_list.remove('.DS_Store')

if '.DS_Store' in TEMP_list:
    TEMP_list.remove('.DS_Store')

if '.ipynb_checkpoints' in EDA_list:
    EDA_list.remove('.ipynb_checkpoints')
    
if '.ipynb_checkpoints' in IBI_list:
    IBI_list.remove('.ipynb_checkpoints')

if '.ipynb_checkpoints' in TEMP_list:
    TEMP_list.remove('.ipynb_checkpoints')

In [10]:
for i in range(len(EDA_list)):
    EDA_route = './KEMDy20/EDA_DATA/' + EDA_list[i]
    IBI_route = './KEMDy20/IBI_DATA/' + IBI_list[i]
    TEMP_route = './KEMDy20/TEMP_DATA/' + TEMP_list[i]
    
    naming_list = EDA_list[i].split('_')
    
    EDA_df = pd.read_csv(EDA_route, index_col=0)
    IBI_df = pd.read_csv(IBI_route, index_col=0)
    TEMP_df = pd.read_csv(TEMP_route, index_col=0)
    
    EDA_df.fillna('', inplace=True)
    IBI_df.fillna('', inplace=True)
    TEMP_df.fillna('', inplace=True)
    
    EDA_col = EDA_df.columns
    IBI_col = IBI_df.columns
    TEMP_col = TEMP_df.columns
    
    E_L = EDA_df[EDA_col[3]].to_list()
    E_R = EDA_df[EDA_col[4]].to_list()
    I_L = IBI_df[EDA_col[3]].to_list()
    I_R = IBI_df[EDA_col[4]].to_list()
    T_L = TEMP_df[EDA_col[3]].to_list()
    T_R = TEMP_df[EDA_col[4]].to_list()
    
    E_SO = []
    I_SO = []
    T_SO = []
    
    for j, item in enumerate(E_L):
        if E_L[j] != '' and E_R[j] != '':
            E_SO.append(E_L[j])
        else:
            E_SO.append(E_L[j] + E_R[j])
        
    for j, item in enumerate(I_L):
        if I_L[j] != '' and I_R[j] != '':
            I_SO.append(I_L[j])
        else:
            I_SO.append(I_L[j] + I_R[j])
        
    for j, item in enumerate(T_L):
        if T_L[j] != '' and T_R[j] != '':
            T_SO.append(T_L[j])
        else:
            T_SO.append(T_L[j] + T_R[j])
    
    # EDA_df['Script Order'] = EDA_df[EDA_col[3]] + EDA_df[EDA_col[4]]
    EDA_df['Script Order'] = E_SO
    # IBI_df['Script Order'] = IBI_df[IBI_col[3]] + IBI_df[IBI_col[4]]
    IBI_df['Script Order'] = I_SO
    # TEMP_df['Script Order'] = TEMP_df[TEMP_col[3]] + TEMP_df[TEMP_col[4]]
    TEMP_df['Script Order'] = T_SO

    EDA_df = EDA_df[['Current Time', 'Status', EDA_col[2], 'Script Order']]
    IBI_df = IBI_df[['Current Time', 'Status', IBI_col[2], 'Script Order']]
    TEMP_df = TEMP_df[['Current Time', 'Status', TEMP_col[2], 'Script Order']]
    
    ###
    
    merge_df = pd.merge(EDA_df, IBI_df, how='outer', left_on='Current Time', right_on='Current Time')
    merge_df.sort_values('Current Time', inplace=True)
    merge_df.fillna('', inplace=True)
    
    merge_df.reset_index(inplace=True)
    merge_df.drop(['index'], axis=1, inplace=True)
    
    Status_x = merge_df['Status_x'].to_list()
    Status_y = merge_df['Status_y'].to_list()

    Script_Order_x = merge_df['Script Order_x'].to_list()
    Script_Order_y = merge_df['Script Order_y'].to_list()

    Status = []
    Script_Order = []

    for j in range(len(Status_x)):
        if Status_x[j] == Status_y[j]:
            Status.append(Status_x[j])
        else:
            if Status_x[j] == '':
                Status.append(Status_y[j])
            elif Status_y[j] == '':
                Status.append(Status_x[j])
            else:
                Status.append(Status_x[j])

    for j in range(len(Script_Order_x)):
        if Script_Order_x[j] == Script_Order_y[j]:
            Script_Order.append(Script_Order_x[j])
        else:
            if Script_Order_x[j] == '':
                Script_Order.append(Script_Order_y[j])
            elif Script_Order_y[j] == '':
                Script_Order.append(Script_Order_x[j])
            else:
                Script_Order.append(Script_Order_x[j])

    merge_df['Status'] = Status
    merge_df['Script Order'] = Script_Order
    merge_df.drop(['Status_x', 'Status_y', 'Script Order_x', 'Script Order_y'], axis=1, inplace=True)
    merge_df = merge_df[['Current Time', 'Status', EDA_col[2], IBI_col[2], 'Script Order']]
    
    Status = merge_df['Status'].to_list()
    Script_Order = merge_df['Script Order'].to_list()

    for j in range(len(Status)):
        if j > 0 and j < len(Status) - 1:
            if Status[j] == 'stay':
                if Script_Order[j-1] == Script_Order[j+1] and Script_Order[j-1] != '' and Script_Order[j+1] != '':
                    Status[j] = Status[j-1]
                    Script_Order[j] = Script_Order[j-1]
    
    merge_df['Status'] = Status
    merge_df['Script Order'] = Script_Order
    merge_df = merge_df[['Current Time', 'Status', EDA_col[2], IBI_col[2], 'Script Order']]
    
    ###
    
    merge_df = pd.merge(merge_df, TEMP_df, how='outer', left_on='Current Time', right_on='Current Time')
    merge_df.sort_values('Current Time', inplace=True)
    merge_df.fillna('', inplace=True)
    
    merge_df.reset_index(inplace=True)
    merge_df.drop(['index'], axis=1, inplace=True)
    
    Status_x = merge_df['Status_x'].to_list()
    Status_y = merge_df['Status_y'].to_list()

    Script_Order_x = merge_df['Script Order_x'].to_list()
    Script_Order_y = merge_df['Script Order_y'].to_list()

    Status = []
    Script_Order = []

    for j in range(len(Status_x)):
        if Status_x[j] == Status_y[j]:
            Status.append(Status_x[j])
        else:
            if Status_x[j] == '':
                Status.append(Status_y[j])
            elif Status_y[j] == '':
                Status.append(Status_x[j])
            else:
                Status.append(Status_x[j])

    for j in range(len(Script_Order_x)):
        if Script_Order_x[j] == Script_Order_y[j]:
            Script_Order.append(Script_Order_x[j])
        else:
            if Script_Order_x[j] == '':
                Script_Order.append(Script_Order_y[j])
            elif Script_Order_y[j] == '':
                Script_Order.append(Script_Order_x[j])
            else:
                Script_Order.append(Script_Order_x[j])
    
    merge_df['Status'] = Status
    merge_df['Script Order'] = Script_Order
    merge_df.drop(['Status_x', 'Status_y', 'Script Order_x', 'Script Order_y'], axis=1, inplace=True)
    merge_df = merge_df[['Current Time', 'Status', EDA_col[2], IBI_col[2], TEMP_col[2], 'Script Order']]
    
    Status = merge_df['Status'].to_list()
    Script_Order = merge_df['Script Order'].to_list()

    for j in range(len(Status)):
        if j > 0 and j < len(Status) - 1:
            if Status[j] == 'stay':
                if Script_Order[j-1] == Script_Order[j+1] and Script_Order[j-1] != '' and Script_Order[j+1] != '':
                    Status[j] = Status[j-1]
                    Script_Order[j] = Script_Order[j-1]
                    
    merge_df['Status'] = Status
    merge_df['Script Order'] = Script_Order
    merge_df = merge_df[['Current Time', 'Status', EDA_col[2], IBI_col[2], TEMP_col[2], 'Script Order']]
    
    naming = naming_list[1] + '_' + naming_list[2] + '_' + naming_list[3]
    save_route = './KEMDy20/DATA_MERGE/' + naming
    merge_df.to_csv(save_route)